In [1]:
import math
from collections import Counter, defaultdict

In [2]:
def entropy(data):
    target_values = [record[-1] for record in data]
    counts = Counter(target_values)
    total = len(target_values)
    ent = 0.0
    for count in counts.values():
        p = count / total
        ent -= p * math.log2(p)
    return ent

def information_gain(data, feature_index):
    total_entropy = entropy(data)
    total_len = len(data)
    
    subsets = defaultdict(list)
    for record in data:
        subsets[record[feature_index]].append(record)
    
    subset_entropy = 0.0
    for subset in subsets.values():
        p = len(subset) / total_len
        subset_entropy += p * entropy(subset)
    
    return total_entropy - subset_entropy

def majority_class(data):
    target_values = [record[-1] for record in data]
    return Counter(target_values).most_common(1)[0][0]

In [3]:
def id3(data, features):

    target_values = [record[-1] for record in data]
    if len(set(target_values)) == 1:
        return target_values[0]

    if not features:
        return majority_class(data)

    gains = [(feature, information_gain(data, feature)) for feature in features]
    best_feature, best_gain = max(gains, key=lambda x: x[1])
    
    if best_gain == 0:
        return majority_class(data)
    
    tree = {best_feature: {}}
    feature_values = set(record[best_feature] for record in data)
    
    for value in feature_values:
        subset = [record for record in data if record[best_feature] == value]
        if not subset:
            tree[best_feature][value] = majority_class(data)
        else:
            remaining_features = [f for f in features if f != best_feature]
            tree[best_feature][value] = id3(subset, remaining_features)
    return tree

In [5]:
data = [
        ['Sunny', 'Hot', 'High', 'Weak', 'No'],
        ['Sunny', 'Hot', 'High', 'Strong', 'No'],
        ['Overcast', 'Hot', 'High', 'Weak', 'Yes'],
        ['Rain', 'Mild', 'High', 'Weak', 'Yes'],
        ['Rain', 'Cool', 'Normal', 'Weak', 'Yes'],
        ['Rain', 'Cool', 'Normal', 'Strong', 'No'],
        ['Overcast', 'Cool', 'Normal', 'Strong', 'Yes'],
        ['Sunny', 'Mild', 'High', 'Weak', 'No'],
        ['Sunny', 'Cool', 'Normal', 'Weak', 'Yes'],
        ['Rain', 'Mild', 'Normal', 'Weak', 'Yes'],
        ['Sunny', 'Mild', 'Normal', 'Strong', 'Yes'],
        ['Overcast', 'Mild', 'High', 'Strong', 'Yes'],
        ['Overcast', 'Hot', 'Normal', 'Weak', 'Yes'],
        ['Rain', 'Mild', 'High', 'Strong', 'No'],
    ]
    
features = list(range(len(data[0]) - 1))
tree = id3(data, features)
print(tree)

{0: {'Overcast': 'Yes', 'Rain': {3: {'Weak': 'Yes', 'Strong': 'No'}}, 'Sunny': {2: {'Normal': 'Yes', 'High': 'No'}}}}


In [9]:
def gini(data):
    target_values = [record[-1] for record in data]
    counts = Counter(target_values)
    total = len(target_values)
    impurity = 1.0
    for count in counts.values():
        prob = count / total
        impurity -= prob ** 2
    return impurity

def split_data(data, feature_index, value):
    left = [record for record in data if record[feature_index] == value]
    right = [record for record in data if record[feature_index] != value]
    return left, right

def gini_index(left, right):
    total_len = len(left) + len(right)
    gini_left = gini(left)
    gini_right = gini(right)
    weighted_gini = (len(left) / total_len) * gini_left + (len(right) / total_len) * gini_right
    return weighted_gini

def majority_class(data):
    target_values = [record[-1] for record in data]
    return Counter(target_values).most_common(1)[0][0]

In [10]:

def cart(data, features):

    target_values = [record[-1] for record in data]
    if len(set(target_values)) == 1:
        return target_values[0]

    if not features:
        return majority_class(data)
    
    best_feature = None
    best_value = None
    best_gini = 1.0
    best_splits = None

    for feature in features:
        values = set(record[feature] for record in data)
        for val in values:
            left, right = split_data(data, feature, val)
            if not left or not right:
                continue
            gini_val = gini_index(left, right)
            if gini_val < best_gini:
                best_gini = gini_val
                best_feature = feature
                best_value = val
                best_splits = (left, right)
    
    if best_feature is None:
        return majority_class(data)
    
    tree = {'feature': best_feature, 'value': best_value, 'left': None, 'right': None}
    remaining_features = features[:] 
    
    tree['left'] = cart(best_splits[0], remaining_features)
    tree['right'] = cart(best_splits[1], remaining_features)
    
    return tree

In [11]:
data = [
        ['Sunny', 'Hot', 'High', 'Weak', 'No'],
        ['Sunny', 'Hot', 'High', 'Strong', 'No'],
        ['Overcast', 'Hot', 'High', 'Weak', 'Yes'],
        ['Rain', 'Mild', 'High', 'Weak', 'Yes'],
        ['Rain', 'Cool', 'Normal', 'Weak', 'Yes'],
        ['Rain', 'Cool', 'Normal', 'Strong', 'No'],
        ['Overcast', 'Cool', 'Normal', 'Strong', 'Yes'],
        ['Sunny', 'Mild', 'High', 'Weak', 'No'],
        ['Sunny', 'Cool', 'Normal', 'Weak', 'Yes'],
        ['Rain', 'Mild', 'Normal', 'Weak', 'Yes'],
        ['Sunny', 'Mild', 'Normal', 'Strong', 'Yes'],
        ['Overcast', 'Mild', 'High', 'Strong', 'Yes'],
        ['Overcast', 'Hot', 'Normal', 'Weak', 'Yes'],
        ['Rain', 'Mild', 'High', 'Strong', 'No'],
    ]
    
features = list(range(len(data[0]) - 1))
tree = id3(data, features)
print(tree)

{0: {'Overcast': 'Yes', 'Rain': {3: {'Weak': 'Yes', 'Strong': 'No'}}, 'Sunny': {2: {'Normal': 'Yes', 'High': 'No'}}}}


In [12]:
def entropy(data):
    target_values = [record[-1] for record in data]
    counts = Counter(target_values)
    total = len(target_values)
    ent = 0.0
    for count in counts.values():
        p = count / total
        ent -= p * math.log2(p)
    return ent

def split_data(data, feature_index):
    subsets = defaultdict(list)
    for record in data:
        subsets[record[feature_index]].append(record)
    return subsets

def info_gain(data, feature_index):
    total_entropy = entropy(data)
    total_len = len(data)
    
    subsets = split_data(data, feature_index)
    
    subset_entropy = 0.0
    split_info = 0.0
    for subset in subsets.values():
        p = len(subset) / total_len
        subset_entropy += p * entropy(subset)
        split_info -= p * math.log2(p) if p > 0 else 0
    
    gain = total_entropy - subset_entropy
    gain_ratio = gain / split_info if split_info != 0 else 0
    
    return gain_ratio

def majority_class(data):
    target_values = [record[-1] for record in data]
    return Counter(target_values).most_common(1)[0][0]

In [13]:
def c45(data, features):
    target_values = [record[-1] for record in data]
    if len(set(target_values)) == 1:
        return target_values[0]
    
    if not features:
        return majority_class(data)
    
    gains = [(feature, info_gain(data, feature)) for feature in features]
    best_feature, best_gain_ratio = max(gains, key=lambda x: x[1])
    
    if best_gain_ratio == 0:
        return majority_class(data)
    
    tree = {best_feature: {}}
    subsets = split_data(data, best_feature)
    
    for feature_val, subset in subsets.items():
        if not subset:
            tree[best_feature][feature_val] = majority_class(data)
        else:
            remaining_features = [f for f in features if f != best_feature]
            tree[best_feature][feature_val] = c45(subset, remaining_features)
    
    return tree

In [14]:
data = [
        ['Sunny', 'Hot', 'High', 'Weak', 'No'],
        ['Sunny', 'Hot', 'High', 'Strong', 'No'],
        ['Overcast', 'Hot', 'High', 'Weak', 'Yes'],
        ['Rain', 'Mild', 'High', 'Weak', 'Yes'],
        ['Rain', 'Cool', 'Normal', 'Weak', 'Yes'],
        ['Rain', 'Cool', 'Normal', 'Strong', 'No'],
        ['Overcast', 'Cool', 'Normal', 'Strong', 'Yes'],
        ['Sunny', 'Mild', 'High', 'Weak', 'No'],
        ['Sunny', 'Cool', 'Normal', 'Weak', 'Yes'],
        ['Rain', 'Mild', 'Normal', 'Weak', 'Yes'],
        ['Sunny', 'Mild', 'Normal', 'Strong', 'Yes'],
        ['Overcast', 'Mild', 'High', 'Strong', 'Yes'],
        ['Overcast', 'Hot', 'Normal', 'Weak', 'Yes'],
        ['Rain', 'Mild', 'High', 'Strong', 'No'],
    ]
    
features = list(range(len(data[0]) - 1))
tree = id3(data, features)
print(tree)

{0: {'Overcast': 'Yes', 'Rain': {3: {'Weak': 'Yes', 'Strong': 'No'}}, 'Sunny': {2: {'Normal': 'Yes', 'High': 'No'}}}}
